## Preliminary

This notebook will teach you how to build your 1st plugin. A plugin is an uploaded Python script triggered based on an event you define. For instance, you can trigger a plugin when a labeler clicks on submit with the `on_submit` handler.
The plugin should have different methods for the different types of events: at the moment these are *on_submit* and *on_review*. These methods will have a predefined set of parameters: the label submitted & the asset_id of the asset labeled.


Therefore, the skeleton of the plugin should be :

```python
from kili.plugins import PluginCore
from kili.types import Label
from numpy.random import random

def custom_function(label: Label):
    label_id = label.get('id')
    print(f"My custom function for review of label with id {label_id}")

class PluginHandler(PluginCore):
    """
    Custom plugin instance
    """

    def custom_method(self, project_id, label_id):
        #custom methods 
        print('Custom method for  label')
        # Use kili for actions with self.kili
        random_seed = random(1)[0]
        print(random_seed)
        if random_seed > 0.5:
            self.logger.warning('Generating issue')
            self.kili.append_to_issues(
                label_id=label_id,
                project_id=project_id,
                text='Random issue generated for this label'
            )

    def on_review(self, label: Label, asset_id: str) -> None:
        """
        Dedicated handler for Review action 
        """
        custom_function(label)

    def on_submit(self, label: Label, asset_id: str) -> None:
        """
        Dedicated handler for Submit action 
        """
        print('On submit called')

        project_id = self.project_id
        label_id = label.get('id')
        
        self.custom_method(project_id, label_id)
```

**IMPORTANT** : Imports of packages inside the plugin are not permitted at the moment (except for `numpy`, & the various kili types).
Don't hesitate to reach out to add more !

**NB: The plugin capabilities of Kili are under active development, and compatible with version 2.125.2 & superior of kili. don't hesitate to reach out via github or the support to provide feedbacks.**

## Instantiate Kili with your personal API_KEY

In [ ]:
#%pip install kili

from kili.client import Kili
import os

api_endpoint = os.getenv('KILI_API_ENDPOINT')
api_key = os.getenv('KILI_API_KEY')


kili = Kili(api_endpoint=api_endpoint, api_key=api_key)

## Develop your plugin

The first step is to define the functions that will be called when the event is triggered. You will be able to iterate on these functions locally with the help of the next section.
 
This cell should be the content of the `.py` file that you will upload as a plugin at the end.
**This file should define the class `PluginHandler` that will have the proper methods.**
We recommend using a modern IDE like VScode to get type hints and autocompletion on the methods.

In [ ]:
from kili.plugins import PluginCore
from kili.types import Label
from numpy.random import random

def custom_function(label: Label):
    label_id = label.get('id')
    print(f"My custom function for review of label with id {label_id}")

class PluginHandler(PluginCore):
    """
    Custom plugin instance
    """

    def custom_method(self, project_id, label_id):
        #custom methods 
        print('Custom method for  label')
        # Use kili for actions with self.kili
        random_seed = random(1)[0]
        print(random_seed)
        if random_seed > 0.5:
            self.logger.warning('Generating issue')
            self.kili.append_to_issues(
                label_id=label_id,
                project_id=project_id,
                text='Random issue generated for this label'
            )

    def on_review(self, label: Label, asset_id: str) -> None:
        """
        Dedicated handler for Review action 
        """
        custom_function(label)

    def on_submit(self, label: Label, asset_id: str) -> None:
        """
        Dedicated handler for Submit action 
        """
        print('On submit called')

        project_id = self.project_id
        label_id = label.get('id')
        
        self.custom_method(project_id, label_id)


### Testing the plugin locally

This section will allow you to test your plugin locally before uploading it.

In [ ]:
project_id = ''

# instantiate the plugin
my_plugin_instance = PluginHandler(kili, project_id)

def get_label(label_id, project_id):
    """
    Function to get the object Label with the same keys as it will be in the plugin
    """
    label = list(kili.labels(
            project_id=project_id,
            label_id=label_id,
            fields=['id', 'jsonResponse', 'author.id', 'labelType', 'createdAt', 'secondsToLabel']
        ))[0]

    label['authorId'] = label['author']['id']
    del label['author']
    return label

### Test the plugin run

If you already have created a project for testing and there are some labels created, you can use directly their ids and use the following cell. Otherwise, you can follow the notebook *plugins_example.ipynb* to create a new project and upload an asset and an associated label.

In [ ]:
asset_id = ''
label_id = ''
project_id = ''

label = get_label(label_id=label_id, project_id=project_id)

my_plugin_instance.on_submit(label=Label(**label), asset_id=asset_id)

### Test the plugin run on Kili
Once you have debugged your code, you may want to upload it directly onto Kili !!

In [ ]:
path_to_plugin = 'path/to/my/plugin.py'
plugin_name = 'My first kili plugin'

kili.upload_plugin(path_to_plugin, plugin_name)

Plugins needs to be activated on the project you want them to run. Be careful with production project, as your custom workflows or rules will be applied !

In [ ]:
kili.activate_plugin_on_project(plugin_name, project_id=project_id)

## Monitoring the plugin
Important to know is that the creation of the plugin takes some time (around 5 minutes), so after that time the plugin will begin to be run (if labeling events will be triggered on this project) and you will be able to get the logs of the runs. To do that, you can use:

In [ ]:
import json

logs = kili.get_plugin_logs(project_id=project_id, plugin_name=plugin_name)

logs_json = json.loads(logs)
print(json.dumps(logs_json, indent=4))

You are also able to set custom date rules for filtering your logs:

In [ ]:
from datetime import date
from datetime import datetime
dt = date.today()  # You can change this date if needed
start_date = datetime.combine(dt, datetime.min.time())

logs = kili.get_plugin_logs(project_id=project_id, plugin_name=plugin_name, start_date=start_date)

## Managing your plugin
You also have several other methods to manage your plugins and their lifecycle.

In [ ]:
# Get the list of all uploaded plugins in your organization
kili.list_plugins()

In [ ]:
# Update a plugin with new source code
updated_path = 'path/to/updated/file.py'
kili.update_plugin(plugin_name=plugin_name, file_path=updated_path)

In [ ]:
# Deactivate the plugin on a certain project (the plugin can still be active for other projects)
kili.deactivate_plugin_on_project(plugin_name=plugin_name, project_id=project_id)

In [ ]:
# Delete the plugin completely (deactivates automatically the plugin from all projects)
kili.delete_plugin(plugin_name=plugin_name)